### 1. Load data

I'll be using a voting dataset I made.

---

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score

from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.linear_model import Lasso, Ridge, ElasticNet

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor

%matplotlib inline

In [2]:
issues = pd.read_csv('./datasets/city_issues_nlp.csv')

### 2. Create X, Y, and normalize X

**X is num_votes here**. We're doing regression.

---

In [18]:
Y = issues.num_votes

print Y.value_counts()

Y = Y.values

X = issues[[c for c in issues.columns if c != 'num_votes']]

X.shape

2     997
3     487
4     229
5     129
6      53
7      52
8      25
9      16
10     10
11      2
Name: num_votes, dtype: int64


(2000, 545)

In [21]:
Xn = ((X - X.mean())/X.std()).values

### 3. Linear regression

1. Cross-validate a linear regression with 5 folds. 
2. Build a linear regression model with the full X and Y.

---

In [22]:
linreg = LinearRegression()

linreg_scores = cross_val_score(linreg, Xn, Y, cv=5)

print linreg_scores
print np.mean(linreg_scores)

[ -2.36908165e+27  -1.96747091e+27  -3.32523315e+27  -4.48032255e+27
  -1.56378452e+27]
-2.74117855525e+27


### 4. Ridge regression

1. Either use `GridSearchCV` or `RidgeCV` to find the best `C` or `alpha` respectively. **Remember that bigger alphas means stronger regularization, and smaller Cs are stronger regularization. (C is the inverse of alpha).**
2. Cross-validate the R2 with Ridge using your optimal C or alpha.
3. Build a final Ridge model and fit it on the full X and Y as you did above.

---

In [32]:
# ridge1 = Ridge(alpha=1)
# ridge2 = Ridge(alpha=10)
# ridge3 = Ridge(alpha=100)
# ridge4 = Ridge(alpha=1000)
# ridge5 = Ridge(alpha=10000)

# for r in [ridge1, ridge2, ridge3, ridge4, ridge5]:
#     print np.mean(cross_val_score(r,Xn, Y, cv=5))
    
ridge = Ridge()

ridge_params = {
    'alpha' : [1,10,1e2,1e3,1e4]
}

ridge_gs = GridSearchCV(ridge,         # 1. Model
                        ridge_params,  # 2. Dictionary With Parameters
                        cv = 5,        # 3. Number of folds for cross validation
                       verbose=3)      # Opt - Have verbose printout describing what param was tested, score, and time     

ridge_gs.fit(Xn, Y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] alpha=1 .........................................................
[CV] ............................... alpha=1, score=-0.543528 -   0.1s
[CV] alpha=1 .........................................................
[CV] ............................... alpha=1, score=-0.265833 -   0.0s
[CV] alpha=1 .........................................................
[CV] ............................... alpha=1, score=-0.392043 -   0.0s
[CV] alpha=1 .........................................................
[CV] ............................... alpha=1, score=-0.334386 -   0.0s
[CV] alpha=1 .........................................................
[CV] ............................... alpha=1, score=-0.136983 -   0.0s
[CV] alpha=10 ........................................................
[CV] .............................. alpha=10, score=-0.321968 -   0.0s
[CV] alpha=10 ........................................................
[CV] ............

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    1.0s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [1, 10, 100.0, 1000.0, 10000.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=3)

In [30]:
print ridge_gs.best_params_
print ridge_gs.best_score_

{'alpha': 1000.0}
0.275599411289


### 5. Lasso regression

1. Use either `GridSearchCV` or `LassoCV` to find the optimal `C` or `alpha` for the Lasso regression.
2. Cross-validate the R2 with Lasso using your optimal C or alpha.
3. Build a final Lasso model fit on the full X and Y.

---

In [37]:
lasso = Lasso()

lasso_params = {
    'alpha' : np.logspace(-4,5,50)
}

lasso_gs = GridSearchCV(lasso,
                       lasso_params,
                       cv=5,
                       verbose=3)

lasso_gs.fit(Xn,Y)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] alpha=0.0001 ....................................................


/Users/sebozek/anaconda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


[CV] .......................... alpha=0.0001, score=-0.582223 -   0.8s
[CV] alpha=0.0001 ....................................................
[CV] .......................... alpha=0.0001, score=-0.314930 -   0.8s
[CV] alpha=0.0001 ....................................................
[CV] .......................... alpha=0.0001, score=-0.405264 -   0.9s
[CV] alpha=0.0001 ....................................................
[CV] .......................... alpha=0.0001, score=-0.332461 -   0.7s
[CV] alpha=0.0001 ....................................................
[CV] .......................... alpha=0.0001, score=-0.166205 -   0.7s
[CV] alpha=0.000152641796718 .........................................
[CV] ............... alpha=0.000152641796718, score=-0.534206 -   0.7s
[CV] alpha=0.000152641796718 .........................................
[CV] ............... alpha=0.000152641796718, score=-0.284117 -   0.7s
[CV] alpha=0.000152641796718 .........................................
[CV] .

[Parallel(n_jobs=1)]: Done  31 tasks       | elapsed:   23.6s


[CV] ................ alpha=0.00126485521686, score=-0.044006 -   1.0s
[CV] alpha=0.00126485521686 ..........................................
[CV] ................ alpha=0.00126485521686, score=-0.148445 -   0.3s
[CV] alpha=0.00126485521686 ..........................................
[CV] ................ alpha=0.00126485521686, score=-0.118647 -   0.7s
[CV] alpha=0.00126485521686 ..........................................
[CV] ................. alpha=0.00126485521686, score=0.044374 -   0.6s
[CV] alpha=0.00193069772888 ..........................................
[CV] ................ alpha=0.00193069772888, score=-0.066186 -   0.3s
[CV] alpha=0.00193069772888 ..........................................
[CV] ................. alpha=0.00193069772888, score=0.025976 -   0.6s
[CV] alpha=0.00193069772888 ..........................................
[CV] ................ alpha=0.00193069772888, score=-0.055578 -   0.4s
[CV] alpha=0.00193069772888 ..........................................
[CV] .

[Parallel(n_jobs=1)]: Done 127 tasks       | elapsed:   34.6s


[CV] alpha=3.90693993705 .............................................
[CV] ................... alpha=3.90693993705, score=-0.000515 -   0.0s
[CV] alpha=3.90693993705 .............................................
[CV] ................... alpha=3.90693993705, score=-0.004898 -   0.0s
[CV] alpha=5.96362331659 .............................................
[CV] ................... alpha=5.96362331659, score=-0.001184 -   0.0s
[CV] alpha=5.96362331659 .............................................
[CV] ................... alpha=5.96362331659, score=-0.002855 -   0.0s
[CV] alpha=5.96362331659 .............................................
[CV] ................... alpha=5.96362331659, score=-0.001006 -   0.0s
[CV] alpha=5.96362331659 .............................................
[CV] ................... alpha=5.96362331659, score=-0.000515 -   0.0s
[CV] alpha=5.96362331659 .............................................
[CV] ................... alpha=5.96362331659, score=-0.004898 -   0.0s
[CV] a

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:   38.7s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([  1.00000e-04,   1.52642e-04,   2.32995e-04,   3.55648e-04,
         5.42868e-04,   8.28643e-04,   1.26486e-03,   1.93070e-03,
         2.94705e-03,   4.49843e-03,   6.86649e-03,   1.04811e-02,
         1.59986e-02,   2.44205e-02,   3.72759e-02,   5.68987e-02,
         8....    1.20679e+04,   1.84207e+04,   2.81177e+04,   4.29193e+04,
         6.55129e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=3)

In [38]:
print lasso_gs.best_params_
print lasso_gs.best_score_

{'alpha': 0.056898660290182992}
0.315248725458


In [44]:
# Can see how many coefficients are zero
print np.sum(lasso_gs.best_estimator_.coef_ == 0) / float(Xn.shape[0])

# count = 0
# for coef in lasso_gs.best_estimator_.coef_:
#     if coef == 0:
#         count += 1
# print float(count) / Xn.shape[0]

# Sometimes can sum off the 

0.2485


### 6. ElasticNet regression

Now you'll get to try out the ElasticNet. It is a combination of the Ridge and the Lasso to leverage the benefits of both!

Arguments to optimize:

    alpha : same as the Ridge/Lasso above
    l1_ratio: this is the proportion of Ridge vs Lasso that the model is. 
        An l1_ratio of 0.0 is a pure Ridge
        An l1_ratio of 1.0 is a pure Lasso
        
1. Use `GridSearchCV` or `ElasticNetCV` to search for the optimal `alpha` and `l1_ratio`. 
2. Explain the probable reason why the it chose the parameters it did as the best ones.
3. Cross-validate the R2 with the ElasticNet using the optimal parameters.
4. Fit the ElasticNet on all X and Y.

---

In [47]:
elastic = ElasticNet()

elastic_params = {
    'alpha' : np.logspace(-4,5,50),
    'l1_ratio' : [0.25,0.5,0.75] 
}

elastic_gs = GridSearchCV(elastic,
                         elastic_params,
                         cv=5)

In [48]:
elastic_gs.fit(Xn, Y)

GridSearchCV(cv=5, error_score='raise',
       estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([  1.00000e-04,   1.52642e-04,   2.32995e-04,   3.55648e-04,
         5.42868e-04,   8.28643e-04,   1.26486e-03,   1.93070e-03,
         2.94705e-03,   4.49843e-03,   6.86649e-03,   1.04811e-02,
         1.59986e-02,   2.44205e-02,   3.72759e-02,   5.68987e-02,
         8....   2.81177e+04,   4.29193e+04,
         6.55129e+04,   1.00000e+05]), 'l1_ratio': [0.25, 0.5, 0.75]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

### 7. DecisionTreeRegressor

1. Use `GridSearchCV` to find the best `max_features`, `max_depth`, and `min_samples_leaf`. Read the documentation and think about what range of parameters would be good to search for each!
2. Cross-validate the R2 as above.
3. Fit a DecisionTreeRegressor on all X and Y as above.

---

### 8. BaggingRegressor

Now we'll use bagging with the DecisionTreeRegressor. Yes, regressions can be done with bagging too!

---

Remember that with the Bagging regressor you first have to initialize the internal "base estimator" that it will copy:

```python
dtr = DecisionTreeRegressor()
```

A cool thing to note is that you can actually gridseach over the internal base estimators as well. So, not only are you finding the best parameters for the BaggingRegressor but also the DecisionTreeRegressors that it copies inside:

```python
bag_params = {
    'base_estimator__max_features':[None],
    'base_estimator__max_depth':[None],
    'base_estimator__min_samples_leaf':[1],
    'max_features': [0.33, 0.66, 0.99],
    'max_samples': [0.1, 0.2, 0.4, 0.6, 0.8, 0.9],
    'n_estimators': [100]
}
```

**Be careful putting too many parameters into the `GridSearchCV`! It can really explode the possible permutations!**

That being said, you'll probably be able to put a decent amount of parameters in since the wine dataset doesn't have many columns.

Next you initialize the BaggingRegressor, putting the desired model as the first argument:

```python
bag = BaggingRegressor(dtr)
```

This tells the BaggingRegressor that you want it to spawn DecisionTreeRegressors as it's internal "children" base estimators.

Lastly, you'll put the BaggingClassifier into the grid searcher and fit on the data (it will cross-validate with the specified `cv` folds.

```python
bag_gs = GridSearchCV(bag, bag_params, cv=5, verbose=1)
bag_gs.fit(X, Y)
```

---

As before...

1. Use `GridSearchCV` to find the best `BaggingRegressor` and optionally internal `DecisionTreeRegressor` parameters.
2. Cross-validate the R2.
3. Fit a `BaggingRegressor` on all X and Y with the optimal parameters.

---

### 9. Get feature importances from `RandomForestRegressor`

The `RandomForestRegressor` has an attribute called `.feature_importances_`. These are the importances of your predictors as measured by how useful they were to the base estimators.

As you may recall, the `RandomForestRegressor` is just a special case of the `BaggingRegressor` that specifically uses decision trees. In fact, you've already done it above. The difference is that this class gives us the feature importances whereas the "generalized" bagging regressor class does not.

1. Save the column names X in a variable.
2. Grid search optimal parameters for the `RandomForestRegressor`.
3. Fit a `RandomForestRegressor` using the optimal parameters you found on the full X and Y.
4. Get out the feature importances.
5. Create a pandas DataFrame where one column is the feature importances and the other column is the X column names.
6. Sort the dataframe you made by feature importances in descending order.
7. Plot the feature importances.

---

In [ ]:
sns.set(style="whitegrid")

f, ax = plt.subplots(figsize=(6, 15))

sns.set_color_codes("muted")
sns.barplot(x="importance", y="feature", data=feature_importances,
            label="feature importances", color="b")

sns.despine(left=True, bottom=True)

plt.show()

### 10. [BONUS] Use a different regression class with the `BaggingRegressor`

You could try `Ridge`, `Lasso`, `ElasticNet`, `SVC`, `KNeighborsRegressor` or any kind of regression class you're interested in!

---